# Custom Gates

## Phase Gate Example

This tutorial shows how to extend `quasar` with user-specified custom gates with parameters (parameter-free custom gates should usually be built with the `U1` and `U2` static methods of class `Gate`).

In [ ]:
import numpy as np
import collections
import quasar
backend = quasar.QuasarSimulatorBackend()

Here we will build a custom one-qubit gate to apply a parametrized phase $\exp(-i \theta)$ with parameter $\theta$ to the $|1\rangle$ state, and leave the $|0\rangle$ qubit state unaltered. To help with that, `quasar` provides functionality for the user to specify the fine details of a `Gate` through the `__init__` function:

In [ ]:
help(quasar.Gate.__init__)

The main deliverable is to write a function that takes an `OrderedDict` of params and returns the $2^N \times 2^N$ `np.ndarray` "$\hat U$" of `dtype=np.complex128` (the unitary matrix defining the gate operation) for a given set of parameters. For instance:

In [ ]:
def fancy_phase_gate(theta=0.0):
    
    return quasar.Gate(
        nqubit=1,
        operator_function=lambda x : np.array([[1.0, 0.0], [0.0, np.exp(-1.j * x['theta'])]], dtype=np.complex128),
        parameters=collections.OrderedDict([('theta', theta)]),
        name='P',
        ascii_symbols=['P'],
        )

Now we can use this phase gate, in concert with the `add_gate` method of `Circuit`:

In [ ]:
circuit = quasar.Circuit().H(0).add_gate(fancy_phase_gate(theta=0.4), 0)
print(circuit)

And check that it acts as intended:

In [ ]:
print(backend.run_statevector(circuit))

If this gate is to be used many times, it might be convenient to add it to the `Circuit` class:

In [ ]:
def _circuit_fancy_phase_gate(
    self,
    qubit,
    theta=0.0,
    **kwargs):
    
    return self.add_gate(
        gate=fancy_phase_gate(theta=theta),
        qubits=(qubit,),
        **kwargs)
quasar.Circuit.fancy_phase = _circuit_fancy_phase_gate

In [ ]:
circuit = quasar.Circuit().H(0).fancy_phase(0, theta=0.4)
print(circuit)

## SO(4) Gate Example

In a more-extended example, we will build a composite 2-qubit gate to cover all rotations in $SO(4)$ according to the description in https://arxiv.org/pdf/1203.0722.pdf (Figure 1).

In [ ]:
import numpy as np
import collections
import quasar

In terms of more conventional gates, the $SO(4)$ gate can be written as 6x $R_y$ gates (each with parameter $\theta$) and 2x CNOT gates.

In [ ]:
circuit1 = quasar.Circuit()
circuit1.Ry(0).Ry(1).CX(0,1).Ry(0).Ry(1).CX(0,1).Ry(0).Ry(1)
print(circuit1)
print('')
print(circuit1.parameter_str)

But, it would get old typing all that out over and over again if we built a larger circuit with many $SO(4)$ gates in it. One solution is to declare a recipe for a new `Gate` that directly implements the action of the whole circuit above (another solution is to use the `add_circuit` function - see the notebook on circuit composition for this). To help with that, we have provided functionality for the user to specify the fine details of a `Gate` through the `__init__` function:

In [ ]:
help(quasar.Gate.__init__)

The main deliverable is to write a function that takes an `OrderedDict` of params and returns the $2^N \times 2^N$ `np.ndarray` "$\hat U$" of `dtype=np.complex128` (the unitary matrix defining the gate operation) for a given set of parameters. For instance:

In [ ]:
def composite_so4_u(params):
    
    theta1 = params['theta1']
    theta2 = params['theta2']
    theta3 = params['theta3']
    theta4 = params['theta4']
    theta5 = params['theta5']
    theta6 = params['theta6']
        
    U12 = np.kron(quasar.Matrix.Ry(theta=theta1), quasar.Matrix.Ry(theta=theta2))
    U34 = np.kron(quasar.Matrix.Ry(theta=theta3), quasar.Matrix.Ry(theta=theta4))
    U56 = np.kron(quasar.Matrix.Ry(theta=theta5), quasar.Matrix.Ry(theta=theta6))
        
    U = np.dot(quasar.Matrix.CX, U12)
    U = np.dot(U34, U)
    U = np.dot(quasar.Matrix.CX, U)
    U = np.dot(U56, U)
        
    return U

We then write a method to build a custom $SO(4)$ gate, which calls the `Gate` `__init__` method with the $\hat U$ function of the previous block, initial parameters, and a few other attributes declaring size `N`, gate name `name`, and a list of ASCII symbols `ascii_symbols` to use in displaying ASCII circuit diagrams:

In [ ]:
def composite_so4(
    theta1=0.0,
    theta2=0.0,
    theta3=0.0,
    theta4=0.0,
    theta5=0.0,
    theta6=0.0,
    ):
    
    params = collections.OrderedDict([
        ('theta1', theta1),
        ('theta2', theta2),
        ('theta3', theta3),
        ('theta4', theta4),
        ('theta5', theta5),
        ('theta6', theta6),
    ])
    
    return quasar.Gate(
        nqubit=2,
        operator_function=composite_so4_u,
        parameters=params,
        name='SO4',
        ascii_symbols=['SO4A', 'SO4B']
        ) 

We can now build a much simpler circuit with just 1x composite $SO(4)$ gate:

In [ ]:
circuit2 = quasar.Circuit()
circuit2.add_gate(times=0, qubits=(0,1), gate=composite_so4())
print(circuit2)
print('')
print(circuit2.parameter_str)

You can see the impact of the `ascii_symbols` flag in the output of the `print(circuit2)` statement. Note that the order of parameters of `circuit1` and `circuit2` are logically equivalent. This means we can generate an iterable list of test parameters:

In [ ]:
theta = 2.0 * np.pi * np.random.rand(6)
print(theta)

And then call `set_parameter_values` with these parameters for both `circuit1` and `circuit2`. Statevector simulation then indicates that the circuits are functionally equivalent:

In [ ]:
circuit1.set_parameter_values(theta)
circuit2.set_parameter_values(theta)
statevector1 = backend.run_statevector(circuit1)
statevector2 = backend.run_statevector(circuit2)
print('statevector1: %s' % statevector1)
print('statevector2: %s' % statevector2)
print('Fidelity: %24.16E' % np.abs(np.dot(statevector1.conj(), statevector2)**2))

Using the new composite gate can make for shorter codes, can make it easier to set certain parameters, and may improve simulation runtimes (as less gate operations are performed). 

Advanced users should consider providing the `adjoint_function` or marking the gate as `involutary` to provide efficient recipes for the `Gate`-level adjoint of a given custom `Gate` object.